In [1]:
from helpers import *
from save_and_load import *
from unet_model import *


/Users/Skolebruker/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# This file is used to produce a deliverable prediction

Select what weights to use, what to call output file etc. Then run all cells

### Set parameters 

In [2]:
#treshold for setting a patch value to 1
foreground_treshold = 0.25
weights = 'weights.hdf5'
filename = "predictions.csv"
#Set to true if you want to save road predictions as image files
save_predicted_images = True
predict_path = "prediction/"

### Make a model and load previously made weights

In [3]:
model = unet()
model.load_weights(weights)

/Users/Skolebruker/Documents/GitHub/MLproject/unet_model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


### Load test images into a list and convert list to numpy array

In [4]:
#load the test images
test_imgs = load_test_images()
#make patches with the same size as original images
test_imgs = make_patches(test_imgs, 608, 608)

### Perform road segmentation using unet

In [5]:
#You may need to change this slice.
prediction = model.predict(test_imgs)

### If requested, predicted images are saved

In [6]:
if(save_predicted_images):
    save_result(predict_path,prediction)


/Users/Skolebruker/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))


### Slice predicted image into patches with size 16x16 and convert these patches to either 1 or 0 

In [7]:
#convert an image with predicted roads to patches
delivery = make_patches(prediction, 16,16)
#set patches to 1 or 0 based on mean color value
delivery = np.asarray([value_to_class(np.mean(delivery[i]), foreground_treshold) for i in range(len(delivery))])
#reshape in order to deliver
delivery = np.reshape(delivery, (-1,38,38))

### Save results to deliverable csv file

In [8]:
#submit for report               
save_submission(delivery, filename)